# 4. Inference

In [1]:
import sys, os
import torch
import pandas as pd
sys.path.append("../../")
sys.path.append("../../../")
sys.path.append("../../../../")
import setting

from Clust.clust.ML.common.common import p1_integratedDataSaving as p1
from Clust.clust.ML.tool import data as ml_data
from Clust.clust.ML.tool import scaler as ml_scaler
from Clust.clust.ML.tool import clean as ml_clean
import pathSetting

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"{device}" " is available.")

/Users/kim-youngkee/Documents/CLUSTER/.clust/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


cpu is available.


In [2]:
# Set regression mode i.e., 'regression','forecast' 
mode_selection = "forecast"

### 4-1. Get model meta by mongodb

In [3]:
from Clust.clust.ingestion.mongo.mongo_client import MongoClient
from Clust.setting import influx_setting_KETI as ins

mongo_client = MongoClient(ins.CLUSTMetaInfo2)

db_name = 'model'
collection_name = 'meta'

all_model_meta = mongo_client.get_all_document(db_name, collection_name)

In [4]:
search = {'modelName': 'TestGRU_rg'}
model_meta= mongo_client.get_document_by_json(db_name, collection_name, search)
model_meta = model_meta[-2]

In [5]:
model_meta

{'trainDataInfo': {'startTime': '2021-02-01 00:00:00',
  'endTime': '2021-03-10 00:00:00',
  'dataInfo': [['farm_swine_air', 'HS2'],
   ['weather_outdoor_keti_clean', 'sangju'],
   ['life_additional_Info', 'trigonometicInfoByHours']],
  'processParam': {'refine_param': {'removeDuplication': {'flag': False},
    'staticFrequency': {'flag': False, 'frequency': None}},
   'outlier_param': {'certainErrorToNaN': {'flag': False},
    'unCertainErrorToNaN': {'flag': False, 'param': {}}},
   'imputation_param': {'flag': False,
    'imputation_method': [],
    'totalNonNanRatio': 80}},
  'integration_freq_sec': 300,
  'cleanParam': 'NoClean',
  'DataSaveMode': 'CSV'},
 'modelName': 'TestGRU_rg',
 'featureList': ['Temperature', 'out_temp', 'sin_hour'],
 'target': 'Temperature',
 'trainDataType': 'timeseries',
 'modelPurpose': 'forecast',
 'model_method': 'GRU_rg',
 'modelTags': ['aaaaa'],
 'cleanTrainDataParam': 'NoClean',
 'NaNProcessingParam': {'feature_cycle': 'Day',
  'feature_cycle_times': 

### 4-2. Inference data selection

In [6]:
# 0. pick only one data
DataMeta = p1.read_json_data(pathSetting.DataMetaPath)
dataList =  list(DataMeta.keys())

In [7]:
dataList

['a5d2781fb26f1520fcee266873deeaa6_4d7849735730d38f0780953d5d57490b',
 'a00e9931e8fe54b78d610e464ca4a4e8_4d7849735730d38f0780953d5d57490b',
 'NoCleanHS1Train',
 'NoCleanHS1Test',
 'CleanHS1Test',
 'CleanHS1Train',
 'TempHS1NoCleanTrain',
 'TempHS1NoCleanTest',
 'HS1NoCleanTrain1',
 'HS1NoCleanTest1',
 'trainClean_Hs1SwineFarmWithWeatherTime',
 'trainNoClean_Hs1SwineFarmWithWeatherTime',
 'testClean_Hs1SwineFarmWithWeatherTime',
 'testNoClean_Hs1SwineFarmWithWeatherTime',
 'CO2HS2CleanForecasting',
 'CO2HS2NoCleanForecasting',
 'CleanHS2Train',
 'NoCleanHS2Train',
 'CleanHS2Test',
 'NoCleanHS2Test']

In [8]:
if mode_selection == 'regression':
    # dataX

    dataName_X = dataList[2]
    dataSaveMode_X = DataMeta[dataName_X]["integrationInfo"]["DataSaveMode"]

    dataFolderName = "data_integrated_result"
    current = os.getcwd()
    dataFolderPath = os.path.join(current, dataFolderName)

    window_num =144
    dataX = ml_data.get_saved_integrated_data(dataSaveMode_X, dataName_X, dataFolderPath)[:window_num]

elif mode_selection == 'forecast':
    LearningModeList = ['train', 'test']
    LearningMode = LearningModeList[1]

    cleanParamList = ['Clean', 'NoClean']
    cleanMode = cleanParamList[1]

    datasetNameList = ['Hs1SwineFarmWithWeatherTime', 'gunwiStrawberryWithWeatherTime', 'strawberryOpenTime']
    datasetName = datasetNameList[0]

    dataName_X = LearningMode + cleanMode + '_' + datasetName
    dataSaveMode_X = DataMeta[dataName_X]['integrationInfo']['DataSaveMode']

    dataX = ml_data.get_saved_integrated_data(dataSaveMode_X, dataName_X, pathSetting.dataFolderPath)
    integration_freq_sec = DataMeta[dataName_X]['integrationInfo']['integration_freq_sec']

In [9]:
print(dataX)

                                CO2ppm    H2Sppm   Humidity      NH3ppm  \
datetime                                                                  
2021-03-10 00:00:00+00:00  3991.666667  3.000000  64.866667  142.000000   
2021-03-10 00:05:00+00:00  4039.500000  3.000000  65.000000  142.000000   
2021-03-10 00:10:00+00:00  4066.000000  3.666667  64.966667  143.666667   
2021-03-10 00:15:00+00:00  4046.500000  3.500000  64.750000  144.000000   
2021-03-10 00:20:00+00:00  4036.000000  3.333333  64.433333  144.666667   
...                                ...       ...        ...         ...   
2021-03-16 22:40:00+00:00  3278.333333  3.333333  62.733333  128.666667   
2021-03-16 22:45:00+00:00  3290.500000  3.000000  62.800000  128.500000   
2021-03-16 22:50:00+00:00  3290.333333  3.333333  62.633333  127.666667   
2021-03-16 22:55:00+00:00  3285.000000  3.500000  62.300000  126.500000   
2021-03-16 23:00:00+00:00  3263.000000  3.000000  62.000000  127.000000   

                        

### 4-3. Inference data preparation

In [24]:
feature_list = model_meta['featureList']
target = model_meta['target']
scaler_param = model_meta['scalerParam']
transform_param = model_meta["transformParameter"]
model_file_path = model_meta['files']['modelFile']['filePath']
model_method = model_meta['model_method']
# train_parameter = model_meta["trainParameter"]

if mode_selection == 'regression':
    # Scaling Inference Input
    X_scaler_file_path = model_meta['files']['XScalerFile']["filePath"]
    y_scaler_file_path = model_meta['files']['yScalerFile']["filePath"]
    
    infer_X, scaler_X = ml_scaler.get_scaled_test_data(dataX[feature_list], X_scaler_file_path, scaler_param)
    scaler_y = ml_scaler.get_scaler_file(y_scaler_file_path)

elif mode_selection == 'forecast':
    clean_param = model_meta['cleanTrainDataParam']
    nan_processing_param = model_meta['NaNProcessingParam']

    # Scaling Inference Input
    X_scaler_file_path = model_meta['files']['XScalerFile']['filePath']

    # only for forecast data inference?
    past_step = transform_param['past_step']
    feature_data = dataX[feature_list]
    step_data = feature_data[-past_step:][feature_list].values
    df_data = pd.DataFrame(step_data, columns=feature_list)

    # infer_X, scaler_X = ml_scaler.get_scaled_test_data(dataX[feature_list], X_scaler_file_path, scaler_param)
    infer_X, scaler_X = ml_scaler.get_scaled_test_data(df_data[feature_list], X_scaler_file_path, scaler_param)
    clean_infer_X = ml_clean.get_cleand_data(infer_X, clean_param, integration_freq_sec, nan_processing_param)

In [27]:
print(clean_infer_X.shape)

(24, 3)


In [28]:
# transform
if mode_selection == 'regression':

    from Clust.clust.transformation.type.DFToNPArray import transDFtoNP, trans_df_to_np, trans_df_to_np_inf
    inferX = trans_df_to_np_inf(infer_X)

# forecast
elif mode_selection == 'forecast':
    
    from Clust.clust.transformation.purpose.machineLearning import LSTMData
    LSTMD = LSTMData()
    inferX, _ = LSTMD.transform_Xy_arr(clean_infer_X, transform_param, transform_param['clean_param'])

1. Original Data Lenagh: 22
2. Removed Data Length: 0
3. Clean Data Leangth: 0


In [29]:
print(type(inferX), inferX.shape)
print(inferX)

<class 'numpy.ndarray'> (0,)
[]


In [14]:
modelParameter = model_meta["modelParameter"]

inferParameter = {
    'device': 'cpu',
    'batch_size': 16
}

In [15]:
# 4. Inference
from Clust.clust.ML.regression_YK.inference import RegressionInference as RI

ri = RI()
ri.set_param(inferParameter)
ri.set_model(model_method, model_file_path, modelParameter)
ri.set_data(inferX)
preds = ri.inference()
print(preds)

inference data shape: torch.Size([1980, 24, 3])

Start inference

tensor([[[ 3.7339e-01,  1.7112e-02, -8.6596e-17,  ...,  4.2275e-01,
          -2.0321e-02,  1.0676e-01],
         [ 4.3348e-01, -2.5668e-02,  1.2201e-01,  ...,  4.9356e-01,
          -5.9893e-02,  2.2565e-01],
         [ 5.1073e-01, -6.4171e-02,  2.3986e-01,  ...,  5.9657e-01,
          -9.4118e-02,  3.3934e-01]],

        [[ 3.7768e-01,  1.1765e-02,  1.5251e-02,  ...,  4.3348e-01,
          -2.5668e-02,  1.2201e-01],
         [ 4.4850e-01, -3.1016e-02,  1.3726e-01,  ...,  5.1073e-01,
          -6.4171e-02,  2.3986e-01],
         [ 5.2575e-01, -6.8449e-02,  2.5407e-01,  ...,  5.9657e-01,
          -9.8396e-02,  3.5355e-01]],

        [[ 3.9056e-01,  6.4171e-03,  3.0502e-02,  ...,  4.4850e-01,
          -3.1016e-02,  1.3726e-01],
         [ 4.5494e-01, -3.6364e-02,  1.5251e-01,  ...,  5.2575e-01,
          -6.8449e-02,  2.5407e-01],
         [ 5.4077e-01, -7.2727e-02,  2.6828e-01,  ...,  5.9657e-01,
          -1.0267e-01,

RuntimeError: shape '[16, -1, 24]' is invalid for input of size 864

In [ ]:
# for regression
if mode_selection == 'regression':
    if scaler_param =='scale':
        base_df_for_inverse = pd.DataFrame(columns=target, index=range(len(preds)))
        base_df_for_inverse[target] = preds
        prediction_result = pd.DataFrame(scaler_y.inverse_transform(base_df_for_inverse), columns=target, index=base_df_for_inverse.index)
    else:
        prediction_result = pd.DataFrame(data={"value": preds}, index=range(len(preds)))

# for forecast
elif mode_selection == 'forecast':
    if scaler_param =='scale':
        base_df_for_inverse = pd.DataFrame(columns=feature_list, index=range(len(preds)))
        base_df_for_inverse[target] = preds
        inverse_result = pd.DataFrame(scaler_X.inverse_transform(base_df_for_inverse), columns=feature_list, index=base_df_for_inverse.index)
        target_data = inverse_result[target]
        prediction_result = pd.DataFrame(data={target: target_data}, index=range(len(preds)))
    else:
        prediction_result = pd.DataFrame(data={target: preds}, index=range(len(preds)))

In [ ]:
prediction_result

,Temperature
0,22.257359
